### *Import Libraries and Dataset**

> Indented block



In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("drive/MyDrive/BankChurners.csv",encoding='latin-1')
df = df.iloc[:, :21]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CLIENTNUM                 10127 non-null  int64  
 1   Attrition_Flag            10127 non-null  object 
 2   Customer_Age              10127 non-null  int64  
 3   Gender                    10127 non-null  object 
 4   Dependent_count           10127 non-null  int64  
 5   Education_Level           10127 non-null  object 
 6   Marital_Status            10127 non-null  object 
 7   Income_Category           10127 non-null  object 
 8   Card_Category             10127 non-null  object 
 9   Months_on_book            10127 non-null  int64  
 10  Total_Relationship_Count  10127 non-null  int64  
 11  Months_Inactive_12_mon    10127 non-null  int64  
 12  Contacts_Count_12_mon     10127 non-null  int64  
 13  Credit_Limit              10127 non-null  float64
 14  Total_

The dataset consists of 5 Categorical features. They are Gender, Education_Level, Marital_Status, Income_Category, Card_Category

In [ ]:
bank_df = df.copy()

In [ ]:
bank_df['Attrition_Flag'] = bank_df['Attrition_Flag'].map({'Attrited Customer': 1, 'Existing Customer': 0})

In [ ]:
bank_df['Attrition_Flag'].value_counts()

0    8500
1    1627
Name: Attrition_Flag, dtype: int64

In [ ]:
bank_num = bank_df.select_dtypes(exclude = ["object"])
bank_num.head(2)

,CLIENTNUM,Attrition_Flag,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,768805383,0,45,3,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,818770008,0,49,5,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105


In [ ]:
bank_num.shape

(10127, 16)

In [ ]:
bank_category = bank_df.select_dtypes(include = ["object"])
bank_category.head(2)

,Gender,Education_Level,Marital_Status,Income_Category,Card_Category
0,M,High School,Married,$60K - $80K,Blue
1,F,Graduate,Single,Less than $40K,Blue


In [ ]:
bank_category = pd.get_dummies(bank_category)
bank_category.head(2)

,Gender_F,Gender_M,Education_Level_College,Education_Level_Doctorate,Education_Level_Graduate,Education_Level_High School,Education_Level_Post-Graduate,Education_Level_Uneducated,Education_Level_Unknown,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Unknown,Income_Category_$120K +,Income_Category_$40K - $60K,Income_Category_$60K - $80K,Income_Category_$80K - $120K,Income_Category_Less than $40K,Income_Category_Unknown,Card_Category_Blue,Card_Category_Gold,Card_Category_Platinum,Card_Category_Silver
0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0
1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0


In [ ]:
bank_category.shape

(10127, 23)

In [ ]:
bank = pd.concat([bank_num, bank_category], axis=1)

In [ ]:
bank.head(2)

,CLIENTNUM,Attrition_Flag,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Gender_F,Gender_M,Education_Level_College,Education_Level_Doctorate,Education_Level_Graduate,Education_Level_High School,Education_Level_Post-Graduate,Education_Level_Uneducated,Education_Level_Unknown,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Unknown,Income_Category_$120K +,Income_Category_$40K - $60K,Income_Category_$60K - $80K,Income_Category_$80K - $120K,Income_Category_Less than $40K,Income_Category_Unknown,Card_Category_Blue,Card_Category_Gold,Card_Category_Platinum,Card_Category_Silver
0,768805383,0,45,3,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0
1,818770008,0,49,5,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0


## **Feature Selection**

###**Random Forest**

In [ ]:
X = bank.iloc[:, 2:]
Y = bank.iloc[:, 1]

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=X.shape[1])
rf.fit(train_X,train_Y)

rf_support = rf.get_support()
rf_support

array([ True, False, False,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [ ]:
rf_feature = X.loc[:,rf_support].columns.tolist()

In [ ]:
rf_feature

['Customer_Age',
 'Total_Relationship_Count',
 'Contacts_Count_12_mon',
 'Credit_Limit',
 'Total_Revolving_Bal',
 'Avg_Open_To_Buy',
 'Total_Amt_Chng_Q4_Q1',
 'Total_Trans_Amt',
 'Total_Trans_Ct',
 'Total_Ct_Chng_Q4_Q1',
 'Avg_Utilization_Ratio']

In [ ]:
print(str(len(rf_feature)), 'selected features')

11 selected features


### **RFE**

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

In [ ]:
X_norm = MinMaxScaler().fit_transform(train_X)

In [ ]:
rfe = RFE(estimator=LogisticRegression(), step=10, verbose=5)
rfe.fit(X_norm, train_Y)
rfe_support = rfe.get_support()
rfe_feature = X.loc[:,rfe_support].columns.tolist()
print(str(len(rfe_feature)), 'selected features')

Fitting estimator with 37 features.
Fitting estimator with 27 features.
18 selected features


In [ ]:
rfe_feature

['Customer_Age',
 'Dependent_count',
 'Total_Relationship_Count',
 'Months_Inactive_12_mon',
 'Contacts_Count_12_mon',
 'Credit_Limit',
 'Total_Revolving_Bal',
 'Total_Amt_Chng_Q4_Q1',
 'Total_Trans_Amt',
 'Total_Trans_Ct',
 'Total_Ct_Chng_Q4_Q1',
 'Gender_M',
 'Education_Level_Post-Graduate',
 'Marital_Status_Married',
 'Income_Category_$120K +',
 'Card_Category_Blue',
 'Card_Category_Gold',
 'Card_Category_Platinum']

### **Lasso**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
X_norm = MinMaxScaler().fit_transform(train_X)

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

lr_selector = SelectFromModel(LogisticRegression(penalty="l2"), max_features=X.shape[1])
lr_selector.fit(X_norm, train_Y)

lr_support = lr_selector.get_support()
lr_feature = X.loc[:, lr_support].columns.tolist()
print(str(len(lr_feature)), 'selected features')

8 selected features


In [ ]:
lr_feature

['Total_Relationship_Count',
 'Months_Inactive_12_mon',
 'Contacts_Count_12_mon',
 'Total_Revolving_Bal',
 'Total_Amt_Chng_Q4_Q1',
 'Total_Trans_Amt',
 'Total_Trans_Ct',
 'Total_Ct_Chng_Q4_Q1']

### **Feature Selection**

In [ ]:
X = bank.iloc[:, 2:]
Y = bank.iloc[:, 1]

In [ ]:
features = pd.DataFrame({'feature': X.columns, 'RF': rf_support, 'RFE': rfe_support, 'lasso': lr_support })

In [ ]:
features['total'] = np.sum(features, axis = 1)

In [ ]:
#feature_selection_df.index = range(1, len(feature_selection_df)+1)

In [ ]:
features = features.sort_values(['total'], ascending= False)
features.reset_index(inplace = True)
features

,index,feature,RF,RFE,lasso,total
0,3,Total_Relationship_Count,True,True,True,3
1,5,Contacts_Count_12_mon,True,True,True,3
2,7,Total_Revolving_Bal,True,True,True,3
3,9,Total_Amt_Chng_Q4_Q1,True,True,True,3
4,10,Total_Trans_Amt,True,True,True,3
5,11,Total_Trans_Ct,True,True,True,3
6,12,Total_Ct_Chng_Q4_Q1,True,True,True,3
7,0,Customer_Age,True,True,False,2
8,4,Months_Inactive_12_mon,False,True,True,2
9,6,Credit_Limit,True,True,False,2


In [ ]:
features = features['feature'][0:10]

In [ ]:
features

0    Total_Relationship_Count
1       Contacts_Count_12_mon
2         Total_Revolving_Bal
3        Total_Amt_Chng_Q4_Q1
4             Total_Trans_Amt
5              Total_Trans_Ct
6         Total_Ct_Chng_Q4_Q1
7                Customer_Age
8      Months_Inactive_12_mon
9                Credit_Limit
Name: feature, dtype: object

## **Train Model**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
X = bank.iloc[:, 2:]
Y = bank.iloc[:, 1]

In [ ]:
#X = X[features]
X.head()

,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Gender_F,Gender_M,Education_Level_College,Education_Level_Doctorate,Education_Level_Graduate,Education_Level_High School,Education_Level_Post-Graduate,Education_Level_Uneducated,Education_Level_Unknown,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Unknown,Income_Category_$120K +,Income_Category_$40K - $60K,Income_Category_$60K - $80K,Income_Category_$80K - $120K,Income_Category_Less than $40K,Income_Category_Unknown,Card_Category_Blue,Card_Category_Gold,Card_Category_Platinum,Card_Category_Silver
0,45,3,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0
1,49,5,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0
2,51,3,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0
3,40,4,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0
4,40,3,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0


In [ ]:
from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler()
#scaler.fit(X)
#X = scaler.transform(X)
# df['min max'] = X.reshape(1,-1)[0]

In [ ]:
train_X, test_X, train_Y, test_Y =  train_test_split(X, Y, test_size=0.3, random_state= 123)

## **Sampling Techniques**

In [ ]:
train_X =train_X[features]

### **SMOTE**

In [ ]:
from collections import Counter

In [ ]:
from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
smote = SMOTE()

In [ ]:
# fit predictor and target variable
train_X,train_Y = smote.fit_resample(train_X, train_Y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
counter = Counter(train_Y)
print(counter)

Counter({0: 6801, 1: 6801})


In [ ]:
train_X = pd.DataFrame(train_X)
train_X.head()

,0,1,2,3,4,5,6,7,8,9
0,1.0,3.0,1728.0,0.595,8554.0,99.0,0.678,3.0,3723.0,54.0
1,1.0,3.0,1803.0,0.493,2107.0,39.0,0.393,4.0,5396.0,58.0
2,6.0,3.0,1648.0,0.732,1436.0,36.0,1.250,1.0,15987.0,45.0
3,4.0,4.0,2517.0,1.158,2616.0,46.0,1.300,3.0,3625.0,34.0
4,5.0,4.0,1926.0,0.602,3806.0,61.0,0.794,3.0,2720.0,49.0


In [ ]:
train_Y = pd.DataFrame(train_Y)
train_Y.head()

,0
0,0
1,1
2,0
3,0
4,0


### **Borderline-SMOTE**

In [ ]:
from collections import Counter

In [ ]:
from imblearn.over_sampling import BorderlineSMOTE

In [ ]:
smote = BorderlineSMOTE()

In [ ]:
# fit predictor and target variable
X,Y = smote.fit_resample(X,Y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
counter = Counter(Y)
print(counter)

Counter({0: 8500, 1: 8500})


In [ ]:
X = pd.DataFrame(X)
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36
0,45.0,3.0,39.0,5.0,1.0,3.0,12691.0,777.0,11914.0,1.335,1144.0,42.0,1.625,0.061,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,49.0,5.0,44.0,6.0,1.0,2.0,8256.0,864.0,7392.0,1.541,1291.0,33.0,3.714,0.105,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,51.0,3.0,36.0,4.0,1.0,0.0,3418.0,0.0,3418.0,2.594,1887.0,20.0,2.333,0.000,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,40.0,4.0,34.0,3.0,4.0,1.0,3313.0,2517.0,796.0,1.405,1171.0,20.0,2.333,0.760,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,40.0,3.0,21.0,5.0,1.0,0.0,4716.0,0.0,4716.0,2.175,816.0,28.0,2.500,0.000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
Y = pd.DataFrame(Y)
Y.head()

,0
0,0
1,0
2,0
3,0
4,0


### **Borderline-SMOTE-SVM**

In [ ]:
from collections import Counter

In [ ]:
from imblearn.over_sampling import SVMSMOTE

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
smote = SVMSMOTE()

In [ ]:
# fit predictor and target variable
train_X, train_Y = smote.fit_resample(train_X, train_Y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

In [ ]:
counter = Counter(train_Y)
print(counter)

Counter({0: 5928, 1: 5928})


In [ ]:
train_X = pd.DataFrame(train_X)
train_X.head()

,0,1,2,3,4,5,6,7,8,9
0,5.0,1.0,1704.0,0.819,4334.0,80.0,0.739,49.0,1.0,8162.0
1,3.0,3.0,848.0,0.762,1408.0,23.0,1.091,43.0,3.0,14881.0
2,4.0,4.0,0.0,0.954,1960.0,48.0,0.600,45.0,6.0,16611.0
3,1.0,2.0,2389.0,0.427,1563.0,36.0,0.440,49.0,2.0,10195.0
4,5.0,0.0,1910.0,0.921,1531.0,35.0,0.667,52.0,3.0,3085.0


In [ ]:
train_Y = pd.DataFrame(train_Y)
train_Y.head()

,0
0,0
1,0
2,0
3,1
4,0


### **ADASYN**

In [ ]:
from collections import Counter

In [ ]:
from imblearn.over_sampling import ADASYN

In [ ]:
X = bank.iloc[:, 2:]
Y = bank.iloc[:, 1]

In [ ]:
smote = ADASYN()

In [ ]:
# fit predictor and target variable
X,Y = smote.fit_resample(X,Y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
counter = Counter(Y)
print(counter)

Counter({1: 8861, 0: 8500})


In [ ]:
X = pd.DataFrame(X)
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36
0,45.0,3.0,39.0,5.0,1.0,3.0,12691.0,777.0,11914.0,1.335,1144.0,42.0,1.625,0.061,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,49.0,5.0,44.0,6.0,1.0,2.0,8256.0,864.0,7392.0,1.541,1291.0,33.0,3.714,0.105,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,51.0,3.0,36.0,4.0,1.0,0.0,3418.0,0.0,3418.0,2.594,1887.0,20.0,2.333,0.000,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,40.0,4.0,34.0,3.0,4.0,1.0,3313.0,2517.0,796.0,1.405,1171.0,20.0,2.333,0.760,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,40.0,3.0,21.0,5.0,1.0,0.0,4716.0,0.0,4716.0,2.175,816.0,28.0,2.500,0.000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
Y = pd.DataFrame(Y)
Y.head()

,0
0,0
1,0
2,0
3,0
4,0


### **Without data balancing**

In [ ]:
X = bank.iloc[:, 2:]
Y = bank.iloc[:, 1]

In [ ]:
X.head(2)

In [ ]:
Y.head(2)

In [ ]:
X.shape, Y.shape

kernel PCA, Backward feature elimination, correlation matrix, random jungle.

## **Classification**

## **Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
LR = LogisticRegression()
LR.fit(train_X, train_Y);

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
predictLR = LR.predict(test_X)

In [ ]:
from sklearn.metrics import precision_score, recall_score
precision = precision_score(predictLR, test_Y)
recall = recall_score(predictLR, test_Y)
print("Precision: ", precision, "\nRecall: ", recall)

Precision:  0.8436830835117773 
Recall:  0.4353591160220994


In [ ]:
print('Logistic Regression: \n', classification_report(predictLR, test_Y))
print('Logistic Regression Accuracy: ', accuracy_score(predictLR, test_Y))

Logistic Regression: 
               precision    recall  f1-score   support

           0       0.80      0.97      0.88      2134
           1       0.84      0.44      0.57       905

    accuracy                           0.81      3039
   macro avg       0.82      0.70      0.73      3039
weighted avg       0.81      0.81      0.79      3039

Logistic Regression Accuracy:  0.8078315235274761


## **KNN**

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)

In [ ]:
k_range = list(range(1, 31))
param_grid = dict(n_neighbors=k_range)
print(param_grid)

{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]}


In [ ]:
grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy')

In [ ]:
grid.fit(train_X, train_Y)

In [ ]:
grid.best_score_

0.9294099230829437

In [ ]:
#Predict the response for test dataset
knn_pred = grid.predict(test_X)

In [ ]:
from sklearn.metrics import precision_score, recall_score
precision = precision_score(knn_pred, test_Y)
recall = recall_score(knn_pred, test_Y)
print("Precision: ", precision, "\nRecall: ", recall)

Precision:  0.6616702355460385 
Recall:  0.5469026548672566


In [ ]:
print('KNN: \n', classification_report(knn_pred, test_Y))
print('KNN: ', accuracy_score(knn_pred, test_Y))

KNN: 
               precision    recall  f1-score   support

           0       0.90      0.94      0.92      2474
           1       0.66      0.55      0.60       565

    accuracy                           0.86      3039
   macro avg       0.78      0.74      0.76      3039
weighted avg       0.86      0.86      0.86      3039

KNN:  0.8637709772951628


## **Random Forest**

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [ ]:
RandomForestClassifier()

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
rf_model = rf.fit(train_X, train_Y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


In [ ]:
rf_train = rf.predict(train_X)
rf_pred = rf.predict(test_X[features])

In [ ]:
rf_train = pd.DataFrame(rf_train)
rf_pred = pd.DataFrame(rf_pred)

In [ ]:
from sklearn.metrics import precision_score, recall_score
precision = precision_score(rf_pred, test_Y)
recall = recall_score(rf_pred, test_Y)
print("Precision: ", precision, "\nRecall: ", recall)

Precision:  0.9079229122055674 
Recall:  0.8379446640316206


In [ ]:
print('RF: \n', classification_report(rf_pred, test_Y))
print('RF: ', accuracy_score(rf_pred, test_Y))

RF: 
               precision    recall  f1-score   support

           0       0.97      0.98      0.98      2533
           1       0.91      0.84      0.87       506

    accuracy                           0.96      3039
   macro avg       0.94      0.91      0.92      3039
weighted avg       0.96      0.96      0.96      3039

RF:  0.9588680487002303


## **XRT**

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
xrt = ExtraTreesClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

In [ ]:
xrt_model = xrt.fit(train_X, train_Y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


In [ ]:
xrt_pred = xrt_model.predict(test_X[features])

In [ ]:
from sklearn.metrics import precision_score, recall_score
precision = precision_score(xrt_pred, test_Y)
recall = recall_score(xrt_pred, test_Y)
print("Precision: ", precision, "\nRecall: ", recall)

Precision:  0.8950749464668094 
Recall:  0.865424430641822


In [ ]:
print('XRT: \n', classification_report(xrt_pred, test_Y))
print('XRT: ', accuracy_score(xrt_pred, test_Y))

XRT: 
               precision    recall  f1-score   support

           0       0.97      0.98      0.98      2556
           1       0.90      0.87      0.88       483

    accuracy                           0.96      3039
   macro avg       0.93      0.92      0.93      3039
weighted avg       0.96      0.96      0.96      3039

XRT:  0.9624876604146101


## **Adaboost**

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
adab_model = AdaBoostClassifier(random_state=1)
adab_model.fit(train_X, train_Y)
adab_train = adab_model.predict(train_X)
adab_pred= adab_model.predict(test_X[features])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
adab_train = pd.DataFrame(adab_train)
adab_pred = pd.DataFrame(adab_pred)

In [ ]:
from sklearn.metrics import precision_score, recall_score
precision = precision_score(adab_pred, test_Y)
recall = recall_score(adab_pred, test_Y)
print("Precision: ", precision, "\nRecall: ", recall)

Precision:  0.9079229122055674 
Recall:  0.7765567765567766


In [ ]:
print('Adaboost: \n', classification_report(adab_pred, test_Y))
print('Adaboost: ', accuracy_score(adab_pred, test_Y))

Adaboost: 
               precision    recall  f1-score   support

           0       0.95      0.98      0.97      2493
           1       0.91      0.78      0.84       546

    accuracy                           0.95      3039
   macro avg       0.93      0.88      0.90      3039
weighted avg       0.94      0.95      0.94      3039

Adaboost:  0.945705824284304


## **GBM**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gbm_model = GradientBoostingClassifier(learning_rate= 0.01, random_state=1)
gbm_model.fit(train_X, train_Y)
gbm_pred= gbm_model.predict(test_X[features])

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
from sklearn.metrics import precision_score, recall_score
precision = precision_score(gbm_pred, test_Y)
recall = recall_score(gbm_pred, test_Y)
print("Precision: ", precision, "\nRecall: ", recall)

Precision:  0.9250535331905781 
Recall:  0.5333333333333333


In [ ]:
print('GBM: \n', classification_report(gbm_pred, test_Y))
print('GBM: ', accuracy_score(gbm_pred, test_Y))

GBM: 
               precision    recall  f1-score   support

           0       0.85      0.98      0.91      2229
           1       0.93      0.53      0.68       810

    accuracy                           0.86      3039
   macro avg       0.89      0.76      0.80      3039
weighted avg       0.87      0.86      0.85      3039

GBM:  0.864100032905561


## **Combo**

In [ ]:
pred1 = pd.DataFrame(rf_pred)
pred2 = pd.DataFrame(adab_pred)
pred3 = pd.DataFrame(xrt_pred)

In [ ]:
pred = pd.concat([pred1, pred2, pred3], axis=1)

In [ ]:
pred.head()

,0,0,0
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,1,1,1


In [ ]:
pred['total'] = np.sum(pred, axis=1)

In [ ]:
pred['final_pred'] = np.where(pred['total'] >= 3,  1, 0)

In [ ]:
pred[pred['final_pred']==1]

,0,0,0,total,final_pred
4,1,1,1,3,1
5,1,1,1,3,1
12,1,1,1,3,1
14,1,1,1,3,1
16,1,1,1,3,1
...,...,...,...,...,...
3005,1,1,1,3,1
3008,1,1,1,3,1
3013,1,1,1,3,1
3016,1,1,1,3,1


In [ ]:
from sklearn.metrics import precision_score, recall_score
precision = precision_score(pred['final_pred'], test_Y)
recall = recall_score(pred['final_pred'], test_Y)
print("Precision: ", precision, "\nRecall: ", recall)

Precision:  0.8458244111349036 
Recall:  0.9186046511627907


In [ ]:
print('RF: \n', classification_report(pred['final_pred'], test_Y))
print('RF: ', accuracy_score(pred['final_pred'], test_Y))

RF: 
               precision    recall  f1-score   support

           0       0.99      0.97      0.98      2609
           1       0.85      0.92      0.88       430

    accuracy                           0.96      3039
   macro avg       0.92      0.95      0.93      3039
weighted avg       0.97      0.96      0.97      3039

RF:  0.9647910496873972
